Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [498]:
from random import randint, choice, choices, random
from copy import deepcopy
from lab9_lib import make_problem
import sys

In [499]:
POPULATION_SIZE = 200
OFFSPRING_SIZE = 50
MUTATION_PROBABILITY = 0.7
XOVER_PROBABILITY = 0.3
XOR_PROBABILITY = 0.3
REORDER_PROBABILITY = 0.7
TOURNAMENT_SIZE = 50
GENERATIONS = 1000
PROBLEM_SIZE = [1,2,5,10]
K=1000

In [500]:
from random import sample, shuffle
counter = 0

class Individual():
    id = -1
    fitness=-1
    genotype=[]
    def __init__(self, _fitness, _genotype):
        self.id = ++counter
        self.fitness = _fitness
        self.genotype = _genotype 
        self.myscore = sum(self.genotype[:int(K/2)])

    def __getitem__(self, i):
        return self.genotype[i]
    
    def __setitem__(self, i, v):
        self.genotype[i] = v
    
    def __len__(self):
        return len(self.genotype)

def select_parent(pop):
    pool = choices(pop, k=TOURNAMENT_SIZE)
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    pos = randint(0, K-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = -1
    return offspring

def mutate_N(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    how_many = randint(1, K)
    targets = sample(range(K), k=how_many)
    for t in targets:
        offspring.genotype[t] = 1 - offspring.genotype[t]
    offspring.fitness = -1
    return offspring

def reorder(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    shuffle(offspring.genotype)
    return offspring

def xor(ind1: Individual, ind2: Individual) -> Individual:
    offspring = deepcopy(ind1)
    offspring.genotype = [ind1[i]^ind2[i] for (i, g) in enumerate(ind1.genotype)]
    return offspring

def xover(ind1: Individual, ind2: Individual) -> Individual:
    how_many = randint(1, K)
    targets = sample(range(K), k=how_many)
    offspring = deepcopy(ind1)
    for t in targets:
        offspring[t] = ind2[t] 
    assert len(offspring.genotype) == K
    return offspring

# Generate Population

In [501]:
population = []
for n in range(10):
    g = choices([0, 1], k=K)
    mo = sum(g)
    ind = Individual(-1, g)
    population.append(ind)

In [502]:
for s in PROBLEM_SIZE:
    fitness = make_problem(s)
    pop = deepcopy(population)
    maxF = -1
    result = "Failure"
    
    for p in pop:
        p.fitness = fitness(p)
        if p.fitness>maxF:
            maxF = p.fitness
        
    g=0
    stillNoPerfectIndividual = True
    while g < GENERATIONS and stillNoPerfectIndividual:
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            o = None
            if random() < XOVER_PROBABILITY:
                # xover # add more xovers
                    p1 = select_parent(pop)
                    p2 = select_parent(pop)
                    o = xover(p1, p2)
                    offspring.append(o)
            
            if random() < REORDER_PROBABILITY:
                if o is not None:
                    o = reorder(o)
                else:
                    p = select_parent(pop)
                    o = reorder(p)
            
            if random() < MUTATION_PROBABILITY:
                if o is not None:
                    o = mutate(o)
                else:
                    p = select_parent(pop)
                    o = mutate(p)

            if o is not None:
              offspring.append(o)

        for o in offspring:
            o.fitness = fitness(o.genotype)

            if o.fitness>maxF:
                maxF = o.fitness
            
            if maxF == 1.0:
                stillNoPerfectIndividual=False
                result = "Success"
                break

        if not stillNoPerfectIndividual:
            break

        pop.extend(offspring)
        pop.sort(key=lambda i: i.fitness, reverse=True)
        pop = pop[:POPULATION_SIZE]
    
        g+=1

        print(f"S={s}\t{round(g/GENERATIONS*100, 2)}%\t\tmaxFitness:{round(maxF,2)}\t\tcalls:{fitness.calls}", end="\r")

    print(f"S={s}\t\tresult:{result}\t\tmaxFitness:{round(maxF,2)}\t\tcalls:{fitness.calls}", end="\n")

S=1		result:Success		maxFitness:1.0		calls:3337


KeyboardInterrupt: 